In [15]:
import numpy as np
from numpy import diff
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import cross_val_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
from IPython.display import clear_output
import scipy
print(os.listdir())
%matplotlib inline

['.ipynb_checkpoints', 'Comparing methods for predicting height.ipynb', 'jd_bst.py', 'Similarity comparison (std. dev).ipynb', 'svk_600_60_dt_men_2018.csv', 'svk_600_60_dt_women_2018.csv', 'svk_height_weight_mens_2008_v2.csv', 'svk_height_weight_womens_2018_v2.csv', 'test.html']


In [13]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
pd.set_option('display.max_columns', 70)

C:\Users\James\AppData\Local\Temp\ipykernel_27144\4021082537.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [20]:
df_mens = pd.read_csv('../data/svk_height_weight_mens_2008_v2.csv',sep=',', index_col=0).reset_index(drop=True)
df_womens = pd.read_csv('../data/svk_height_weight_womens_2008_v2.csv',sep=',', index_col=0).reset_index(drop=True)

FileNotFoundError: [Errno 2] No such file or directory: '../data/svk_height_weight_womens_2008_v2.csv'

In [9]:
df_mens.sample(2)

,child_id,ATV_8,ATV_9,ATV_10,ATV_11,ATV_12,ATV_13,ATV_14,ATV_15,ATV_16,ATV_17,ATV_18,ATT_8,ATT_9,ATT_10,ATT_11,ATT_12,ATT_13,ATT_14,ATT_15,ATT_16,ATT_17,ATT_18
6478,1000301911,1380.000000,1437.916667,1499.166667,1560.833333,1640.000000,1749.583333,1830.000000,1865.000000,1867.083333,1874.166667,1886.250000,335.000000,341.25,375.00,430.833333,499.583333,602.916667,690.833333,740.833333,786.666667,818.333333,838.333333
3138,1000234385,1344.166667,1395.833333,1439.416667,1499.833333,1537.083333,1604.166667,1707.166667,1756.333333,1805.833333,1814.583333,1824.166667,296.666667,355.00,401.25,414.166667,433.333333,501.916667,580.416667,676.833333,717.083333,765.833333,796.583333


In [10]:
len(df_mens)

7577

In [11]:
len(df_womens)

9181

In [12]:
atv_column_names = [x for x in df_mens.columns if 'ATV' in x]
df_mens_atv = df_mens[atv_column_names]
atv_column_names = [x for x in df_womens.columns if 'ATV' in x]
df_womens_atv = df_womens[atv_column_names]

In [13]:
df_mens_atv = df_mens_atv[(df_mens_atv != 0).all(1)]

In [14]:
df_womens_atv = df_womens_atv[(df_womens_atv != 0).all(1)]

In [15]:
att_column_names = [x for x in df_mens.columns if 'ATT' in x]
df_mens_att = df_mens[att_column_names]
att_column_names = [x for x in df_womens.columns if 'ATT' in x]
df_womens_att = df_womens[att_column_names]

In [18]:
df_mens_600m = pd.read_csv('../data/svk_600_60_dt_men_2018.csv',sep=',', index_col=0).reset_index(drop=True)
df_womens_600m = pd.read_csv('../data/svk_600_60_dt_women_2018.csv',sep=',', index_col=0).reset_index(drop=True)

In [19]:
att600_column_names = [x for x in df_mens_600m.columns if 'T600' in x]
df_mens_att600 = df_mens_600m[att600_column_names]
att600_column_names = [x for x in df_womens_600m.columns if 'T600' in x]
df_womens_att600 = df_womens_600m[att600_column_names]

In [2]:
atdt_column_names = [x for x in df_mens_600m.columns if 'DT' in x] 
df_mens_atv = df_mens_600m[atdt_column_names]
atdt_column_names = [x for x in df_womens_600m.columns if 'DT' in x]
df_womens_atdt = df_womens_600m[atdt_column_names]

NameError: name 'df_mens_600m' is not defined

In [1]:
print(df_mens_atv)

NameError: name 'df_mens_atv' is not defined

In [21]:
#Because we calculate difefrences between years, we need to name the columns right, so we prepare the names here
atv_column_names_gradient = []
for i in range(8, 18):
    atv_column_names_gradient.append('ATV_diff_'+str(i)+'-'+str(i+1))

# Height men

In [23]:
#Height men
all_all_errors=[]
for start_age in range(8,18):
    print('------------------')
    print('start_age: ' + str(start_age))
    print('------------------')
    col_index_for_start_age = start_age-8 #first year = 8
    diffs = diff(df_mens_atv[df_mens_atv.columns[col_index_for_start_age:]].values)
    atv_column_names_gradient_current = atv_column_names_gradient[col_index_for_start_age:]
    df_grad = pd.DataFrame(data=diffs, columns=atv_column_names_gradient_current)

    all_errors=[]
    all_std_devs=[]
    for age in range(start_age,18):
        #print(age)
        col_index_for_age = age-start_age #first year = 8
        remaining_ages_for_difference = 18-age
        number_of_similar_rows_to_include_for_predictions = 100
        predicting_errors =[]
        #Calculate similarities between year differences
        #Simialrities are calculated only for values lower than age
        #print(col_index_for_start_age)
        #print(col_index_for_age)
        #similarity = cosine_similarity(df_mens_atv[df_mens_atv.columns[col_index_for_start_age:col_index_for_start_age + col_index_for_age+1]])
        similarity = scipy.spatial.distance.cdist(df_mens_atv[df_mens_atv.columns[col_index_for_start_age:col_index_for_start_age + col_index_for_age+1]], df_mens_atv[df_mens_atv.columns[col_index_for_start_age:col_index_for_start_age + col_index_for_age+1]], metric='euclidean')

        #We iterate through rows
        for i in range(len(similarity)):
            row = similarity[i]
            sorted_indexes = list(np.argsort(row)) #We sort values from biggest similarity to lowest-we get indexes of those values
            del sorted_indexes[sorted_indexes.index(i)] #Remove value of index of the same value to remove similati of the same values

            #We get rows with simmilar differences
            similar_rows = []
            for j in range(number_of_similar_rows_to_include_for_predictions):
                similar_rows.append(df_grad.iloc[sorted_indexes[j]])

            #Now we need to calculate remaining differences to estimate value at 18
            next_diferences = [0]*remaining_ages_for_difference
            #print(similar_rows[-1])
            #print()
            for similar_row in similar_rows:
                for k in range(remaining_ages_for_difference):
                    next_diferences[-k-1] = next_diferences[-k-1] + similar_row[-k-1] #we add difference from simmilar rows for each year
                    #print(similar_row[-k-1])
            next_diferences = np.array(next_diferences)/number_of_similar_rows_to_include_for_predictions #divide to get average
            all_next_differences = sum(next_diferences)
            current_value = df_mens_atv.iloc[i][df_mens_atv.columns[col_index_for_start_age + col_index_for_age]]

            predicted_value = current_value + all_next_differences
            actual_value = df_mens_atv.iloc[i][df_mens_atv.columns[-1]]

            predicting_error = abs(predicted_value - actual_value)
            predicting_errors.append(predicting_error)


        #Average predicting error
        average_error = np.median(predicting_errors)
        std_error = np.std(predicting_errors)
        print('Average error: ' + str(average_error))
        print('Std. error: ' + str(std_error))
        all_errors.append(average_error)
        all_std_devs.append(std_error)
    print(all_errors)
    print(all_std_devs)
    print('Sum errors: ' + str(sum(all_errors)))

------------------
start_age: 8
------------------
Average error: 6.2399999999999665
Std. error: 5.738211122556062
Average error: 6.060833333333427
Std. error: 5.723724670515744
Average error: 5.819166666666668
Std. error: 5.698747223202996
Average error: 5.846666666666621
Std. error: 5.4584373076450845
Average error: 5.723333333333429
Std. error: 5.434673128362942
Average error: 5.4316666666667075
Std. error: 5.335349621157837
Average error: 5.31916666666671
Std. error: 5.358354151236677
Average error: 5.138333333333335
Std. error: 5.287664392481464
Average error: 4.654166666666647
Std. error: 4.967100393146613
Average error: 3.4633333333334377
Std. error: 4.194219797860213
[6.2399999999999665, 6.060833333333427, 5.819166666666668, 5.846666666666621, 5.723333333333429, 5.4316666666667075, 5.31916666666671, 5.138333333333335, 4.654166666666647, 3.4633333333334377]
[5.738211122556062, 5.723724670515744, 5.698747223202996, 5.4584373076450845, 5.434673128362942, 5.335349621157837, 5.35835

# Height women

In [24]:
#Height women
all_all_errors=[]
for start_age in range(8,18):
    print('------------------')
    print('start_age: ' + str(start_age))
    print('------------------')
    col_index_for_start_age = start_age-8 #first year = 8
    diffs = diff(df_womens_atv[df_womens_atv.columns[col_index_for_start_age:]].values)
    atv_column_names_gradient_current = atv_column_names_gradient[col_index_for_start_age:]
    df_grad = pd.DataFrame(data=diffs, columns=atv_column_names_gradient_current)

    all_errors=[]
    all_std_devs=[]
    for age in range(start_age,18):
        #print(age)
        col_index_for_age = age-start_age #first year = 8
        remaining_ages_for_difference = 18-age
        number_of_similar_rows_to_include_for_predictions = 100
        predicting_errors =[]
        #Calculate similarities between year differences
        #Simialrities are calculated only for values lower than age
        #print(col_index_for_start_age)
        #print(col_index_for_age)
        #similarity = cosine_similarity(df_womens_atv[df_womens_atv.columns[col_index_for_start_age:col_index_for_start_age + col_index_for_age+1]])
        similarity = scipy.spatial.distance.cdist(df_womens_atv[df_womens_atv.columns[col_index_for_start_age:col_index_for_start_age + col_index_for_age+1]], df_womens_atv[df_womens_atv.columns[col_index_for_start_age:col_index_for_start_age + col_index_for_age+1]], metric='euclidean')

        #We iterate through rows
        for i in range(len(similarity)):
            row = similarity[i]
            sorted_indexes = list(np.argsort(row)) #We sort values from biggest similarity to lowest-we get indexes of those values
            del sorted_indexes[sorted_indexes.index(i)] #Remove value of index of the same value to remove similati of the same values

            #We get rows with simmilar differences
            similar_rows = []
            for j in range(number_of_similar_rows_to_include_for_predictions):
                similar_rows.append(df_grad.iloc[sorted_indexes[j]])

            #Now we need to calculate remaining differences to estimate value at 18
            next_diferences = [0]*remaining_ages_for_difference
            #print(similar_rows[-1])
            #print()
            for similar_row in similar_rows:
                for k in range(remaining_ages_for_difference):
                    next_diferences[-k-1] = next_diferences[-k-1] + similar_row[-k-1] #we add difference from simmilar rows for each year
                    #print(similar_row[-k-1])
            next_diferences = np.array(next_diferences)/number_of_similar_rows_to_include_for_predictions #divide to get average
            all_next_differences = sum(next_diferences)
            current_value = df_womens_atv.iloc[i][df_womens_atv.columns[col_index_for_start_age + col_index_for_age]]

            predicted_value = current_value + all_next_differences
            actual_value = df_womens_atv.iloc[i][df_womens_atv.columns[-1]]

            predicting_error = abs(predicted_value - actual_value)
            predicting_errors.append(predicting_error)


        #Average predicting error
        average_error = np.median(predicting_errors)
        std_error = np.std(predicting_errors)
        print('Average error: ' + str(average_error))
        print('Std. error: ' + str(std_error))
        all_errors.append(average_error)
        all_std_devs.append(std_error)
    print(all_errors)
    print(all_std_devs)
    print('Sum errors: ' + str(sum(all_errors)))

------------------
start_age: 8
------------------
Average error: 28.506666666666547
Std. error: 43.90840195066897
Average error: 27.584423076922803
Std. error: 42.76557982865211
Average error: 27.640454545454418
Std. error: 34.711706140431644
Average error: 27.14125000000047
Std. error: 28.576320892267656
Average error: 24.61000000000024
Std. error: 27.998268727597754
Average error: 16.886474358973942
Std. error: 24.66863904342307
Average error: 11.291856060606165
Std. error: 19.04990974534392
Average error: 8.403216783216067
Std. error: 16.157570759944182
Average error: 6.3865093240087845
Std. error: 16.996213501820908
Average error: 3.9127243589738328
Std. error: 11.396631018710098
[28.506666666666547, 27.584423076922803, 27.640454545454418, 27.14125000000047, 24.61000000000024, 16.886474358973942, 11.291856060606165, 8.403216783216067, 6.3865093240087845, 3.9127243589738328]
[43.90840195066897, 42.76557982865211, 34.711706140431644, 28.576320892267656, 27.998268727597754, 24.668639

KeyboardInterrupt: 

## We need std dev for predicting all next years not just at 18 years old. So we modify this a bit

# Height men

In [ ]:
#Height men
number_of_similar_rows_to_include_for_predictions = 100
all_all_errors=[]
for start_age in range(8,9): #Predpostavimo da imamo vsa leta za nazaj, ker razlike niso tako velike
    print('------------------')
    print('start_age: ' + str(start_age))
    print('------------------')
    col_index_for_start_age = start_age-8 #first year = 8
    diffs = diff(df_mens_atv[df_mens_atv.columns[col_index_for_start_age:]].values)
    atv_column_names_gradient_current = atv_column_names_gradient[col_index_for_start_age:]
    df_grad = pd.DataFrame(data=diffs, columns=atv_column_names_gradient_current)

    all_errors=[]
    all_std_errors = []
    for age in range(start_age,18):
        print('Data up to age: ' + str(age))
        col_index_for_age = age-start_age # start_age = 8
        
        #remaining_ages_for_difference = 18-age
        
        #Calculate similarities between year differences
        #Simialrities are calculated only for values lower than age
        similarity = scipy.spatial.distance.cdist(df_mens_atv[df_mens_atv.columns[col_index_for_start_age:col_index_for_start_age + col_index_for_age+1]], df_mens_atv[df_mens_atv.columns[col_index_for_start_age:col_index_for_start_age + col_index_for_age+1]], metric='euclidean')

        predicting_errors =[]
        all_errors=[]
        all_std_errors = []
        for remaining_ages_for_difference in range(1, 18-age+1): #, 18-age+1):
        #for remaining_ages_for_difference in range(1, 2): #, 18-age+1):
            print(f'Predicting for {remaining_ages_for_difference} years ahead')
            predicting_errors =[]
            #We iterate through rows
            for i in range(len(similarity)):            
                row = similarity[i]
                #print(df_mens_atv.iloc[i][df_mens_atv.columns[col_index_for_start_age + col_index_for_age]])
                #print(df_mens_atv.iloc[i][df_mens_atv.columns[col_index_for_start_age + col_index_for_age + remaining_ages_for_difference]])
                sorted_indexes = list(np.argsort(row)) #We sort values from biggest similarity to lowest-we get indexes of those values
                del sorted_indexes[sorted_indexes.index(i)] #Remove value of index of the same value to remove similati of the same values

                #We get rows with simmilar differences
                similar_rows = []
                for j in range(number_of_similar_rows_to_include_for_predictions):
                    similar_rows.append(df_grad.iloc[sorted_indexes[j]]) #Original version                    

                #print(similar_rows[0:3])
                #print('-------------------')
                
                #Now we need to calculate remaining differences to estimate value at x
                next_diferences = [0]*remaining_ages_for_difference
                #print(similar_rows[-1])
                #print()
                for similar_row in similar_rows:
                    for k in range(1, remaining_ages_for_difference+1):
                        #print('k')
                        #print(col_index_for_start_age + col_index_for_age)
                        #print(k-1)
                        #print(similar_row[k-1])
                        #next_diferences[-k-1] = next_diferences[-k-1] + similar_row[-k-1] #we add difference from simmilar rows for each year
                        next_diferences[k-1] = next_diferences[k-1] + similar_row[col_index_for_start_age + col_index_for_age + k-1] #we add difference from simmilar rows for each year
                        #next_diferences[k] = next_diferences[k] + similar_row[k]
                        #print(f'Similar row value: {similar_row[k]}')
                next_diferences = np.array(next_diferences)/number_of_similar_rows_to_include_for_predictions #divide to get average
                #print('next')
                #print(next_diferences)
                #break
                all_next_differences = sum(next_diferences)
                current_value = df_mens_atv.iloc[i][df_mens_atv.columns[col_index_for_start_age + col_index_for_age]]

                predicted_value = current_value + all_next_differences
                #actual_value = df_mens_atv.iloc[i][df_mens_atv.columns[-1]]
                actual_value = df_mens_atv.iloc[i][df_mens_atv.columns[col_index_for_start_age + col_index_for_age + remaining_ages_for_difference]]

                predicting_error = abs(predicted_value - actual_value)
                predicting_errors.append(predicting_error)


            #Average predicting error
            predicting_errors = [x for x in predicting_errors if x < 150]
            average_error = np.median(predicting_errors)
            std_error = np.std(predicting_errors)
            print('Average error: ' + str(average_error))
            #print(predicting_errors)
            print('Std. error: ' + str(std_error))
            all_errors.append(average_error)
            all_std_errors.append(std_error)
        print(all_errors)
        print(all_std_errors)
        print('Sum errors: ' + str(sum(all_errors)))

# Height women

In [ ]:
#Height women
number_of_similar_rows_to_include_for_predictions = 100
all_all_errors=[]
for start_age in range(8,9): #Predpostavimo da imamo vsa leta za nazaj, ker razlike niso tako velike
    print('------------------')
    print('start_age: ' + str(start_age))
    print('------------------')
    col_index_for_start_age = start_age-8 #first year = 8
    diffs = diff(df_womens_atv[df_womens_atv.columns[col_index_for_start_age:]].values)
    atv_column_names_gradient_current = atv_column_names_gradient[col_index_for_start_age:]
    df_grad = pd.DataFrame(data=diffs, columns=atv_column_names_gradient_current)

    all_errors=[]
    all_std_errors = []
    for age in range(start_age,18):
        print('Data up to age: ' + str(age))
        col_index_for_age = age-start_age # start_age = 8
        
        #remaining_ages_for_difference = 18-age
        
        #Calculate similarities between year differences
        #Simialrities are calculated only for values lower than age
        similarity = scipy.spatial.distance.cdist(df_womens_atv[df_womens_atv.columns[col_index_for_start_age:col_index_for_start_age + col_index_for_age+1]], df_womens_atv[df_womens_atv.columns[col_index_for_start_age:col_index_for_start_age + col_index_for_age+1]], metric='euclidean')

        predicting_errors =[]
        all_errors=[]
        all_std_errors = []
        for remaining_ages_for_difference in range(1, 18-age+1): #, 18-age+1):
        #for remaining_ages_for_difference in range(1, 2): #, 18-age+1):
            print(f'Predicting for {remaining_ages_for_difference} years ahead')
            predicting_errors =[]
            #We iterate through rows
            for i in range(len(similarity)):            
                row = similarity[i]
                #print(df_womens_atv.iloc[i][df_womens_atv.columns[col_index_for_start_age + col_index_for_age]])
                #print(df_womens_atv.iloc[i][df_womens_atv.columns[col_index_for_start_age + col_index_for_age + remaining_ages_for_difference]])
                sorted_indexes = list(np.argsort(row)) #We sort values from biggest similarity to lowest-we get indexes of those values
                del sorted_indexes[sorted_indexes.index(i)] #Remove value of index of the same value to remove similati of the same values

                #We get rows with simmilar differences
                similar_rows = []
                for j in range(number_of_similar_rows_to_include_for_predictions):
                    similar_rows.append(df_grad.iloc[sorted_indexes[j]]) #Original version                    

                #print(similar_rows[0:3])
                #print('-------------------')
                
                #Now we need to calculate remaining differences to estimate value at x
                next_diferences = [0]*remaining_ages_for_difference
                #print(similar_rows[-1])
                #print()
                for similar_row in similar_rows:
                    for k in range(1, remaining_ages_for_difference+1):
                        #print('k')
                        #print(col_index_for_start_age + col_index_for_age)
                        #print(k-1)
                        #print(similar_row[k-1])
                        #next_diferences[-k-1] = next_diferences[-k-1] + similar_row[-k-1] #we add difference from simmilar rows for each year
                        next_diferences[k-1] = next_diferences[k-1] + similar_row[col_index_for_start_age + col_index_for_age + k-1] #we add difference from simmilar rows for each year
                        #next_diferences[k] = next_diferences[k] + similar_row[k]
                        #print(f'Similar row value: {similar_row[k]}')
                next_diferences = np.array(next_diferences)/number_of_similar_rows_to_include_for_predictions #divide to get average
                #print('next')
                #print(next_diferences)
                #break
                all_next_differences = sum(next_diferences)
                current_value = df_womens_atv.iloc[i][df_womens_atv.columns[col_index_for_start_age + col_index_for_age]]

                predicted_value = current_value + all_next_differences
                #actual_value = df_womens_atv.iloc[i][df_womens_atv.columns[-1]]
                actual_value = df_womens_atv.iloc[i][df_womens_atv.columns[col_index_for_start_age + col_index_for_age + remaining_ages_for_difference]]

                predicting_error = abs(predicted_value - actual_value)
                predicting_errors.append(predicting_error)


            #Average predicting error
            predicting_errors = [x for x in predicting_errors if x < 150]
            average_error = np.median(predicting_errors)
            std_error = np.std(predicting_errors)
            print('Average error: ' + str(average_error))
            #print(predicting_errors)
            print('Std. error: ' + str(std_error))
            all_errors.append(average_error)
            all_std_errors.append(std_error)
        print(all_errors)
        print(all_std_errors)
        print('Sum errors: ' + str(sum(all_errors)))